In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
wiki_url = 'https://en.wikipedia.org/wiki/Shark_Tank_India'
#identifying searcahble class on target table
style_val="text-align:center; font-size:85%; line-height:17px; width:auto;"

#assigning response
response = requests.get(wiki_url)
#creating soup object by parsing html data
soup= BeautifulSoup(response.text, 'html.parser')

#finding required table from the parsed data
winning_teams_table = soup.find('table',attrs={'style':style_val})

#converting dataframe, using first index as it returns a list
df = pd.read_html(str(winning_teams_table))[0]

In [3]:
df = df.iloc[:,[0,2,3,4,5]]
ep = list(df['Ep. No.']['Ep. No.'])
brand = list(df['Brand']['Brand'])
idea = list(df['Idea']['Idea'])
deal = list(df['Deal']['Deal'])

df = pd.DataFrame({'Episode':ep,'Brand':brand,'Idea':idea,'Deal':deal})
df.head()

,Episode,Brand,Idea,Deal
0,1,BluePine Industries,Frozen Momos,₹75 lakhs for 18% equity
1,1,Booz scooters,Renting e-bike for mobility in private spaces,₹40 lakhs for 50% equity
2,1,Heart up my Sleeves,Detachable Sleeves,₹25 lakhs for 30% equity
3,2,Tagz Foods,Healthy Potato Chips,₹70 lakhs for 2.75% equity
4,2,Head and Heart,Brain Development Course,No Deal


In [4]:
no_deal=[]
for i in range(len(df)):
    if df['Deal'][i] == 'No Deal':
        no_deal.append(i)
df = df.drop(no_deal).reset_index(drop=True)

In [5]:
df.head()

,Episode,Brand,Idea,Deal
0,1,BluePine Industries,Frozen Momos,₹75 lakhs for 18% equity
1,1,Booz scooters,Renting e-bike for mobility in private spaces,₹40 lakhs for 50% equity
2,1,Heart up my Sleeves,Detachable Sleeves,₹25 lakhs for 30% equity
3,2,Tagz Foods,Healthy Potato Chips,₹70 lakhs for 2.75% equity
4,3,Peeschute,Disposable Urine Bag,₹75 lakhs for 6% equity


In [6]:
df = df.drop([len(df)-1,len(df)-2]).reset_index(drop=True)


In [7]:
for i in range(len(df)):
    if 'lakh' not in df["Deal"][i].lower() and 'crore' not in df["Deal"][i].lower():
        lst = df["Deal"][i].split()
        lst.insert(1,'rupee')
        df["Deal"][i] = " ".join(lst)

In [8]:
for i in range(len(df)):
    if len(df['Deal'][i].split()) > 5:
        new_lst = df['Deal'][i].split()[:5]
        df['Deal'][i] = " ".join(new_lst)
df['Deal'][39] = '₹40 Lakhs for 3% equity'
df = df.drop([len(df)-1]).reset_index(drop=True)

In [9]:
val=[]
in_cr=[]

for i in range(len(df)):
    if 'lakh' in df['Deal'][i].lower():
        if df['Brand'][i] == 'Jain Shikanji':

          num = (int((df["Deal"][i].split())[0][:]))/100
        else:
          num = (int((df["Deal"][i].split())[0][1:]))/100
        pct = float((df["Deal"][i].split())[3][:-1])
        val.append(str(round(num * (100/pct),2)) + " Cr")
        r = round(num * (100/pct),2)
        in_cr.append(r)

    elif 'crore' in df['Deal'][i].lower():
        
        num = float((df["Deal"][i].split())[0][1:])
        pct = float((df["Deal"][i].split())[3][:-1])
        val.append(str(round(num * (100/pct),2)) + " Cr")
        r = round(num * (100/pct),2)
        in_cr.append(r)

    else:        
        num = float((df["Deal"][i].split())[0][1:])
        pct = float((df["Deal"][i].split())[3][:-1])
        val.append(str(round(num * (100/pct),2)) + " Rs")
        r = (num * (100/pct)/10000000)
        in_cr.append(r)

In [10]:
df['Valuation'] = val
df['Valuation in Crore'] = in_cr
df.sort_values(by=['Valuation in Crore'])[::-1].reset_index(drop=True)

,Episode,Brand,Idea,Deal,Valuation,Valuation in Crore
0,5,Revamp Moto,E-Bike,₹1 crore for 1.5% equity,66.67 Cr,66.670000
1,13,Annie,Braille Literary Device,₹1.05 crore at 3% equity,35.0 Cr,35.000000
2,FINALEWEEK,AyuRythm,Ayurvedic Wellness App,₹75 lakhs for 2.68% Equity,27.99 Cr,27.990000
3,2,Tagz Foods,Healthy Potato Chips,₹70 lakhs for 2.75% equity,25.45 Cr,25.450000
4,7,Raising Superstars,Child Development App,₹1 crore for 4% equity,25.0 Cr,25.000000
...,...,...,...,...,...,...
60,1,Booz scooters,Renting e-bike for mobility in private spaces,₹40 lakhs for 50% equity,0.8 Cr,0.800000
61,21,Sid07 Designs,Inventions,₹25 Lakhs for 75% equity,0.33 Cr,0.330000
62,24,KG Agrotech,Agricultural Innovations,₹10 lakhs for 40% Equity,0.25 Cr,0.250000
63,27,Watt Technovations,Ventilated PPE Kits,₹101 rupee for 4% Equity,2525.0 Rs,0.000253
